In [1]:
# 2 weeks together
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [6]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2+7) 

write_folder="/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) 

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2020, 5, 30), datetime.date(2020, 5, 23)]

In [7]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_2_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_1_after_201806=list_1_after_201806+list_2_after_201806
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

3


In [10]:
new_2_biweekly_files = new_2_biweekly_files[:2]
new_2_biweekly_files

['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-05-23/MediaStormMasterWeekly20200526-112101-527.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-05-30/MediaStormMasterWeekly20200602-111902-387.txt']

In [11]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
        print(df_all_new_master.shape)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))


(265181, 3)
(496956, 3)
496956
496953


In [12]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

273


In [13]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_20

In [14]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]

uploaded_new_seg.sort()
uploaded_new_seg=uploaded_new_seg[-24:]

print("len(uploaded_new_seg)"+str(len(uploaded_new_seg)))
previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

len(uploaded_new_seg)24


In [15]:
# Exclude the quarterly regular segments


In [16]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2020-06-12 12:04:53.893186 496953 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:04:54.492088 496953 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-12 12:04:54.795778 496953 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:04:55.099128 496953 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-12 12:04:59.463734 496950 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_

2020-06-12 12:05:27.530474 496923 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:05:27.790053 496923 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-12 12:05:27.906443 496923 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:05:28.064407 496923 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-12 12:05:28.186756 496923 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P

2020-06-12 12:05:40.451852 496916 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:05:40.589945 496916 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-12 12:05:40.802019 496915 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:05:40.993062 496915 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-12 12:05:41.145003 496915 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_P

2020-06-12 12:06:02.768566 496904 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-12 12:06:03.607799 496904 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:03.856059 496904 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-12 12:06:07.782392 496899 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/C_all_quarterly_segments_RFM20Q1.csv
2020-06-12 12:06:08.003006 496899 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RF

2020-06-12 12:06:20.467859 496894 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:20.624677 496894 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-12 12:06:24.134376 496890 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:24.811955 496890 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-12 12:06:26.241236 496888 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H

2020-06-12 12:06:37.499208 496879 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:37.624469 496879 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-12 12:06:37.760481 496879 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:37.881620 496879 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:38.194662 496879 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L

2020-06-12 12:06:50.943153 496872 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:51.098614 496872 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-12 12:06:51.254982 496872 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_S_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:51.693874 496871 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_S_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-12 12:06:51.837872 496871 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_S_Above50K_Singl

2020-06-12 12:06:59.344576 496863 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-08/NewReward_2020-02-08_T.csv
2020-06-12 12:06:59.698375 496863 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_P.csv
2020-06-12 12:06:59.911943 496863 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_S.csv
2020-06-12 12:07:00.204953 496863 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_T.csv
2020-06-12 12:07:00.673271 496863 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-03-07/NewReward_2020-03-07_P.csv
2020-06-12 12:07:00.887650 496863 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-03-07/NewReward_2020-03-07_S.csv
2020-06-12 12:07:01.348418 496863 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-03-07/NewReward_2020-03-07_T.csv
2020-06-12 12:07:01.785993 496862 /home/jian/celery/BiWeekly_New_Sign

In [17]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

496861


In [18]:
zipmap = pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
zipmap=zipmap.parse("unique_zips_full_footprint",dtype=str)
zipmap['zip_type']=zipmap['zip_type'].replace("zips_10","trans_S")

zipmap['zipcodegroup'] = zipmap['zip_type'].apply(lambda x: x.split("_")[1])
print(zipmap.shape)
print(zipmap['zip_cd'].nunique())

zipmap=zipmap.rename(columns={"zip_cd":"customer_zip_code"})

zipmap=zipmap[['customer_zip_code','zipcodegroup']]
zipmap.head(2)

(12980, 3)
12980


,customer_zip_code,zipcodegroup
0,75040,P
1,20743,P


In [19]:
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment_new_signups'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

[nan 'S' 'P']
['T' 'S' 'P']
P 115502
S 48784
T 332575


In [20]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,115502
1,S,48784
2,T,332575


In [21]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment_new_signups
0,fc5ac7da63567df4a7d39d5159c7ca7e5c50ea766bcd9a...,18e971d67452df115df81655b8314e61c4a9e02397728c...,NaN,NewReward_053020_T
2,b84814356370de0964c1989bfd222ae889c6b32d467c2d...,c6f9504f62277b136cd655c3f65d15dd0d52ad27033ee0...,NaN,NewReward_053020_T


In [22]:
df_all_new_master.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,zipcodegroup
0,fc5ac7da63567df4a7d39d5159c7ca7e5c50ea766bcd9a...,18e971d67452df115df81655b8314e61c4a9e02397728c...,NaN,T
1,e14f31f8d0c2b2d489e526b355c7cc8f460455928d2cf9...,5290596e62ef43d90e35a69efaa5d6c7cc927715382a6a...,36587,S


In [23]:
df_all_new_master[pd.isnull(df_all_new_master['customer_zip_code'])].shape

(313837, 4)

In [24]:
# to do below 
print(datetime.datetime.now())

2020-06-12 12:08:40.816929


In [26]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
transport.close()
print("Done: ",datetime.datetime.now())

Done:  2020-06-12 12:10:57.437764


In [27]:
os.getcwd()

'/home/jian/celery/BiWeekly_New_SignUps'